In [ ]:
pip install gns3fy

In [1]:
import gns3fy

In [2]:
server = gns3fy.Gns3Connector("http://localhost:3080")

In [3]:
lab = gns3fy.Project(name="GNS_v0", connector=server)

In [4]:
lab.get()
print(lab)

Project(name='GNS_v0', project_id='f9a30b3e-69d7-4287-b267-f60d3b2b7286', status='opened', path='C:\\Users\\User\\GNS3\\projects\\GNS_v0', filename='GNS_v0.gns3', auto_start=False, auto_close=True, auto_open=False, drawing_grid_size=25, grid_size=75, scene_height=1000, scene_width=2000, show_grid=False, show_interface_labels=True, show_layers=False, snap_to_grid=False, supplier=None, variables=None, zoom=100, stats={'drawings': 0, 'links': 10, 'nodes': 7, 'snapshots': 0}, snapshots=None, drawings=None)


In [6]:
import gns3fy

# Define the server object to establish the connection
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define the lab you want to load and assign the server connector
lab = gns3fy.Project(name="GNS_v0", connector=gns3_server)

# Retrieve its information and display
lab.get()
print(lab)

# Open the project
lab.open()
lab.status

# Verify the stats
lab.stats

# List the names and status of all the nodes in the project
for node in lab.nodes:
    print(f"Node: {node.name} -- Node Type: {node.node_type} -- Status: {node.status}")

Project(name='GNS_v0', project_id='f9a30b3e-69d7-4287-b267-f60d3b2b7286', status='opened', path='C:\\Users\\User\\GNS3\\projects\\GNS_v0', filename='GNS_v0.gns3', auto_start=False, auto_close=True, auto_open=False, drawing_grid_size=25, grid_size=75, scene_height=1000, scene_width=2000, show_grid=False, show_interface_labels=False, show_layers=False, snap_to_grid=False, supplier=None, variables=None, zoom=100, stats={'drawings': 0, 'links': 10, 'nodes': 7, 'snapshots': 0}, snapshots=None, drawings=None)
Node: R1 -- Node Type: dynamips -- Status: started
Node: R2 -- Node Type: dynamips -- Status: started
Node: R3 -- Node Type: dynamips -- Status: started
Node: R4 -- Node Type: dynamips -- Status: started
Node: R5 -- Node Type: dynamips -- Status: started
Node: R6 -- Node Type: dynamips -- Status: started
Node: R7 -- Node Type: dynamips -- Status: started


In [7]:
import gns3fy
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
projects = gns3_server.get_projects()
for project in projects:
    print(project["name"])

GNS_v0
NRP_TP_3_BGP
NRP_TP_3_BGP_v1
untitled


In [12]:
import gns3fy

# Connect to the GNS3 server
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define your project name
project_name = "GNS_v0"
project = gns3fy.Project(name=project_name, connector=gns3_server)
project.get()

# Define network intent: router names, interfaces, and IPv6 addresses
network_intent = {
    "Router1": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::1/64"
        },
        "ripng_interfaces": ["GigabitEthernet1/0"]
    },
    "Router2": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::2/64"
        },
        "ripng_interfaces": ["GigabitEthernet1/0"]
    },
    # Add more routers here
}

# Function to push configurations via Telnet
def configure_router(node, config_commands):
    node.console_start()  # Start the console if not already running
    with node.console() as telnet:
        telnet.read_until(b"Router>")
        telnet.write(b"enable\n")
        telnet.read_until(b"Router#")
        telnet.write(b"configure terminal\n")
        
        # Apply each configuration command
        for command in config_commands:
            telnet.write(f"{command}\n".encode())
        
        telnet.write(b"end\n")
        telnet.write(b"write memory\n")
        telnet.read_until(b"Router#")

# Iterate over routers and apply configurations
for node in project.nodes:
    if node.name in network_intent:
        router_config = []
        intent = network_intent[node.name]
        
        # Configure IPv6 addresses
        for interface, ipv6_address in intent["interfaces"].items():
            router_config.append(f"interface {interface}")
            router_config.append(f" ipv6 address {ipv6_address}")
            router_config.append(" no shutdown")
        
        # Enable RIPng
        router_config.append("ipv6 router rip RIPNG")
        for interface in intent["ripng_interfaces"]:
            router_config.append(f" interface {interface}")
            router_config.append(" ipv6 rip RIPNG enable")
        
        print(f"Configuring {node.name}...")
        configure_router(node, router_config)
        print(f"{node.name} configured successfully!")


In [7]:
import requests

response = requests.get("http://localhost:3080/v2/projects")
if response.status_code == 200:
    print("Server response:", response.json())
else:
    print(f"Error: {response.status_code}, Response: {response.text}")

Server response: [{'auto_close': True, 'auto_open': False, 'auto_start': False, 'drawing_grid_size': 25, 'filename': 'GNS_v0.gns3', 'grid_size': 75, 'name': 'GNS_v0', 'path': 'C:\\Users\\User\\GNS3\\projects\\GNS_v0', 'project_id': 'f9a30b3e-69d7-4287-b267-f60d3b2b7286', 'scene_height': 1000, 'scene_width': 2000, 'show_grid': False, 'show_interface_labels': False, 'show_layers': False, 'snap_to_grid': False, 'status': 'opened', 'supplier': None, 'variables': None, 'zoom': 100}, {'auto_close': True, 'auto_open': False, 'auto_start': False, 'drawing_grid_size': 25, 'filename': 'NRP_TP_3_BGP.gns3', 'grid_size': 75, 'name': 'NRP_TP_3_BGP', 'path': 'C:\\Users\\User\\GNS3\\projects\\NRP_TP_3_BGP', 'project_id': 'be9ce004-2fb6-4d9b-b286-be5da3984f5d', 'scene_height': 1000, 'scene_width': 2000, 'show_grid': False, 'show_interface_labels': True, 'show_layers': False, 'snap_to_grid': False, 'status': 'closed', 'supplier': None, 'variables': None, 'zoom': 100}, {'auto_close': False, 'auto_open': 

In [16]:
import gns3fy 

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

for node in project.nodes :
    print(node.ports)

[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'FastEthernet0/0', 'port_number': 0, 'short_name': 'f0/0'}, {'adapter_number': 1, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet1/0', 'port_number': 0, 'short_name': 'g1/0'}, {'adapter_number': 2, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet2/0', 'port_number': 0, 'short_name': 'g2/0'}, {'adapter_number': 3, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet3/0', 'port_number': 0, 'short_name': 'g3/0'}]
[{'adapter_number': 0, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'FastEthernet0/0', 'port_number': 0, 'short_name': 'f0/0'}, {'adapter_number': 1, 'data_link_types': {'Ethernet': 'DLT_EN10MB'}, 'link_type': 'ethernet', 'name': 'GigabitEthernet1/0', 'port_number': 0, 'short_name': 'g1/0'}, {'adapter_numb

In [1]:
import gns3fy

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

# Loop through nodes properly
for node_id in project.nodes:
    node = project.get_node(node_id)  # Fetch the node by its ID
    print(f"Node {node['name']}")
    for interface in node['interfaces']:
        print(f"  Interface {interface['name']} - {interface.get('connected_node', 'No connection')}")


TypeError: 'NoneType' object is not subscriptable

In [18]:
import gns3fy

gns3_server = gns3fy.Gns3Connector("http://localhost:3080")
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

for node in project.nodes:
    print(f"Node {node.name} ({node.node_type})")

    # Loop through ports (interfaces) of the node
    for port in node.ports:
        connected_node = port.get('connected_node')  # Check if this interface is connected
        if connected_node:
            print(f"  {port['name']} is connected to {connected_node}")
        else:
            print(f"  {port['name']} is not connected")

Node R1 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R2 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R3 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R4 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R5 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not connected
Node R6 (dynamips)
  FastEthernet0/0 is not connected
  GigabitEthernet1/0 is not connected
  GigabitEthernet2/0 is not connected
  GigabitEthernet3/0 is not co

Test de ChatGPT

In [8]:
pip install netmiko

   ---------------------------------------- 0.0/562.3 kB ? eta -:--:--
   --------------------------------------- 562.3/562.3 kB 15.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from netmiko import ConnectHandler

# Full network configuration
topology_config = {
    "R1": {
        "ip": "127.0.0.1",
        "port": 5000,
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::1/64",
            "GigabitEthernet2/0": "2001:db8:2::1/64",
        },
    },
    "R2": {
        "ip": "127.0.0.1",
        "port": 5001,
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::2/64",
            "GigabitEthernet2/0": "2001:db8:3::1/64",
            "GigabitEthernet3/0": "2001:db8:4::1/64",
        },
    },
    "R3": {
        "ip": "127.0.0.1",
        "port": 5002,
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:3::2/64",
            "GigabitEthernet2/0": "2001:db8:2::2/64",
            "GigabitEthernet3/0": "2001:db8:5::1/64",
        },
    },
    "R4": {
        "ip": "127.0.0.1",
        "port": 5003,
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:4::2/64",
            "GigabitEthernet2/0": "2001:db8:6::1/64",
            "GigabitEthernet3/0": "2001:db8:7::1/64",
        },
    },
    "R5": {
        "ip": "127.0.0.1",
        "port": 5004,
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:5::2/64",
            "GigabitEthernet2/0": "2001:db8:7::2/64",
            "GigabitEthernet3/0": "2001:db8:8::1/64",
        },
    },
    "R6": {
        "ip": "127.0.0.1",
        "port": 5005,
        "interfaces": {
            "FastEthernet0/0": "2001:db8:6::2/64",
            "GigabitEthernet1/0": "2001:db8:9::1/64",
        },
    },
    "R7": {
        "ip": "127.0.0.1",
        "port": 5006,
        "interfaces": {
            "FastEthernet0/0": "2001:db8:8::2/64",
            "GigabitEthernet2/0": "2001:db8:9::2/64",
        },
    },
}

# Router credentials
username = "cisco"
password = "cisco"
enable_password = "cisco"

# Function to apply configurations to a router
def configure_router(router_name, router_config):
    device_params = {
        "device_type": "cisco_ios",
        "host": router_config["ip"],
        "port": router_config["port"],
        "username": username,
        "password": password,
        "secret": enable_password,
    }

    try:
        # Connect to the device
        print(f"Connecting to {router_name}...")
        connection = ConnectHandler(**device_params)
        connection.enable()

        # Apply interface configurations
        for interface, ipv6_address in router_config["interfaces"].items():
            commands = [
                f"interface {interface}",
                f"ipv6 address {ipv6_address}",
                "ipv6 enable",
                "ipv6 rip RIPng enable",
                "exit",
            ]
            output = connection.send_config_set(commands)
            print(f"Configuration applied to {router_name} {interface}:\n{output}")

        # Save the configuration
        connection.save_config()
        print(f"Configuration saved on {router_name}.")

        # Close the connection
        connection.disconnect()

    except Exception as e:
        print(f"Failed to configure {router_name}: {e}")

# Iterate through the topology and apply configurations
for router, config in topology_config.items():
    configure_router(router, config)

print("All configurations applied successfully.")


Exception (client): Error reading SSH protocol banner'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\paramiko\transport.py", line 2369, in _check_banner
    buf = self.packetizer.readline(timeout)
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\paramiko\packet.py", line 401, in readline
    return u(buf)
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\paramiko\util.py", line 333, in u
    return s.decode(encoding)
           ~~~~~~~~^^^^^^^^^^
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\paramiko\transport.py", line 2185, in run
    self._check_banner()
    ~~~

Connecting to R1...
Failed to configure R1: 
A paramiko SSHException occurred during connection creation:

Error reading SSH protocol banner'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Connecting to R2...
Failed to configure R2: 
A paramiko SSHException occurred during connection creation:

Error reading SSH protocol banner'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Connecting to R3...
Failed to configure R3: 
A paramiko SSHException occurred during connection creation:

Error reading SSH protocol banner'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Connecting to R4...
Failed to configure R4: 
A paramiko SSHException occurred during connection creation:

Error reading SSH protocol banner'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


Connecting to R5...
Failed to configure R5: 
A paramiko SSHException occurred during connection creation:

Error reading SSH protocol banner'utf-8' 

In [6]:
import gns3fy

# Define GNS3 server connector
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define the project and retrieve information
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

# Open the project
if project.status != "opened":
    project.open()

# Define the network intent
topology_config = {
    "R1": {
        "GigabitEthernet1/0": "2001:db8:1::1/64",
        "GigabitEthernet2/0": "2001:db8:2::1/64",
    },
    "R2": {
        "GigabitEthernet1/0": "2001:db8:1::2/64",
        "GigabitEthernet2/0": "2001:db8:3::1/64",
        "GigabitEthernet3/0": "2001:db8:4::1/64",
    },
    "R3": {
        "GigabitEthernet1/0": "2001:db8:3::2/64",
        "GigabitEthernet2/0": "2001:db8:2::2/64",
        "GigabitEthernet3/0": "2001:db8:5::1/64",
    },
    "R4": {
        "GigabitEthernet1/0": "2001:db8:4::2/64",
        "GigabitEthernet2/0": "2001:db8:6::1/64",
        "GigabitEthernet3/0": "2001:db8:5::2/64",
        "FastEthernet0/0": "2001:db8:7::1/64",
    },
    "R5": {
        "GigabitEthernet1/0": "2001:db8:5::3/64",
        "GigabitEthernet2/0": "2001:db8:8::1/64",
        "GigabitEthernet3/0": "2001:db8:6::2/64",
        "FastEthernet0/0": "2001:db8:9::1/64",
    },
    "R6": {
        "GigabitEthernet1/0": "2001:db8:6::3/64",
        "FastEthernet0/0": "2001:db8:10::1/64",
    },
    "R7": {
        "GigabitEthernet2/0": "2001:db8:8::2/64",
        "FastEthernet0/0": "2001:db8:9::2/64",
    },
}

# Apply the configuration
def apply_config(router_name, interface_configs):
    node = next(node for node in project.nodes if node.name == router_name)
    node.start()  # Ensure the node is running

    # Use Telnet or an external console interface to send commands
    print(f"Configuring {router_name}...")
    for interface, ipv6_address in interface_configs.items():
        commands = [
            f"interface {interface}",
            f"ipv6 address {ipv6_address}",
            "ipv6 enable",
            "ipv6 rip RIPng enable",
            "exit",
        ]
        for command in commands:
            print(f"Would send command: {command}")  # Replace with actual Telnet send

    # Save the configuration
    print("Would send: write memory")  # Replace with actual Telnet send

# Iterate over the topology and configure each router
for router, interfaces in topology_config.items():
    apply_config(router, interfaces)

print("Configuration logic executed successfully.")


Configuring R1...
Would send command: interface GigabitEthernet1/0
Would send command: ipv6 address 2001:db8:1::1/64
Would send command: ipv6 enable
Would send command: ipv6 rip RIPng enable
Would send command: exit
Would send command: interface GigabitEthernet2/0
Would send command: ipv6 address 2001:db8:2::1/64
Would send command: ipv6 enable
Would send command: ipv6 rip RIPng enable
Would send command: exit
Would send: write memory
Configuring R2...
Would send command: interface GigabitEthernet1/0
Would send command: ipv6 address 2001:db8:1::2/64
Would send command: ipv6 enable
Would send command: ipv6 rip RIPng enable
Would send command: exit
Would send command: interface GigabitEthernet2/0
Would send command: ipv6 address 2001:db8:3::1/64
Would send command: ipv6 enable
Would send command: ipv6 rip RIPng enable
Would send command: exit
Would send command: interface GigabitEthernet3/0
Would send command: ipv6 address 2001:db8:4::1/64
Would send command: ipv6 enable
Would send comma

In [7]:
import gns3fy
import telnetlib
import time

# Define GNS3 server connector
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define the project and retrieve information
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()

# Open the project
if project.status != "opened":
    project.open()

# Define the network intent
topology_config = {
    "R1": {
        "GigabitEthernet1/0": "2001:db8:1::1/64",
        "GigabitEthernet2/0": "2001:db8:2::1/64",
    },
    "R2": {
        "GigabitEthernet1/0": "2001:db8:1::2/64",
        "GigabitEthernet2/0": "2001:db8:3::1/64",
        "GigabitEthernet3/0": "2001:db8:4::1/64",
    },
    # Add other routers as needed
}

# Telnet connection details
TELNET_PORT = 5000  # Replace with your GNS3 telnet base port (default: 5000)
TELNET_HOST = "127.0.0.1"

# Function to send commands via Telnet
def send_telnet_commands(router_name, base_port, interface_configs):
    try:
        # Calculate the telnet port for the router (e.g., R1 -> 5000, R2 -> 5001, etc.)
        telnet_port = base_port + int(router_name[1:]) - 1
        print(f"Connecting to {router_name} on port {telnet_port}...")

        # Connect via Telnet
        with telnetlib.Telnet(TELNET_HOST, telnet_port) as tn:
            time.sleep(1)
            tn.read_until(b"Username:", timeout=5)
            tn.write(b"cisco\n")  # Replace with your username
            tn.read_until(b"Password:", timeout=5)
            tn.write(b"cisco\n")  # Replace with your password

            # Enter enable mode
            tn.write(b"enable\n")
            tn.read_until(b"Password:", timeout=5)
            tn.write(b"cisco\n")  # Replace with enable password

            # Enter configuration mode
            tn.write(b"configure terminal\n")
            time.sleep(1)

            # Send interface configurations
            for interface, ipv6_address in interface_configs.items():
                commands = [
                    f"interface {interface}",
                    f"ipv6 address {ipv6_address}",
                    "ipv6 enable",
                    "ipv6 rip RIPng enable",
                    "exit",
                ]
                for command in commands:
                    tn.write(f"{command}\n".encode())
                    time.sleep(0.5)

            # Save configuration
            tn.write(b"write memory\n")
            time.sleep(1)
            tn.write(b"exit\n")
            tn.write(b"exit\n")

            print(f"Configuration applied to {router_name}.")

    except Exception as e:
        print(f"Failed to configure {router_name}: {e}")

# Apply the configuration to all routers
for router, interfaces in topology_config.items():
    send_telnet_commands(router, TELNET_PORT, interfaces)

print("Configuration applied successfully.")


ModuleNotFoundError: No module named 'telnetlib'

In [3]:
import gns3fy

# Connect to the GNS3 server
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Define project and topology
project_name = "GNS_v0"  # Replace with your GNS3 project name
project = gns3fy.Project(name=project_name, connector=gns3_server)
project.get()

# Get the nodes in the project
project_nodes = project.nodes

# Full network configuration for all 7 routers
topology_config = {
    "R1": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::1/64",
            "GigabitEthernet2/0": "2001:db8:2::1/64",
        },
    },
    "R2": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:1::2/64",
            "GigabitEthernet2/0": "2001:db8:3::1/64",
            "GigabitEthernet3/0": "2001:db8:4::1/64",
        },
    },
    "R3": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:3::2/64",
            "GigabitEthernet2/0": "2001:db8:2::2/64",
            "GigabitEthernet3/0": "2001:db8:5::1/64",
        },
    },
    "R4": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:4::2/64",
            "GigabitEthernet2/0": "2001:db8:6::1/64",
            "GigabitEthernet3/0": "2001:db8:7::1/64",
        },
    },
    "R5": {
        "interfaces": {
            "GigabitEthernet1/0": "2001:db8:5::2/64",
            "GigabitEthernet2/0": "2001:db8:7::2/64",
            "GigabitEthernet3/0": "2001:db8:8::1/64",
        },
    },
    "R6": {
        "interfaces": {
            "FastEthernet0/0": "2001:db8:6::2/64",
            "GigabitEthernet1/0": "2001:db8:9::1/64",
        },
    },
    "R7": {
        "interfaces": {
            "FastEthernet0/0": "2001:db8:8::2/64",
            "GigabitEthernet2/0": "2001:db8:9::2/64",
        },
    },
}

# Function to apply configuration to routers
def configure_router(router_name, interfaces):
    # Find the node
    node = next((n for n in project_nodes if n.name == router_name), None)
    if not node:
        print(f"Router {router_name} not found in the project.")
        return

    print(f"Configuring {router_name}...")

    # Start the router if it's not already running
    if node.status != "started":
        node.start()
        print(f"{router_name} started.")

    # Build configuration commands
    config_lines = []
    for interface, ipv6_address in interfaces.items():
        config_lines.extend([
            f"interface {interface}",
            f"ipv6 address {ipv6_address}",
            "ipv6 enable",
            "ipv6 rip RIPng enable",
            "exit",
        ])
    config_lines.append("write memory")  # Save the configuration

    # Send configuration commands to the router
    output = node.console(lines=config_lines)
    print(f"Configuration applied to {router_name}:\n{output}")

# Iterate through the topology and apply configurations
for router_name, config in topology_config.items():
    configure_router(router_name, config["interfaces"])

print("All configurations applied successfully.")


Configuring R1...


TypeError: 'int' object is not callable

In [4]:
import gns3fy

# Connect to the GNS3 server
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Load the project
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()
project.open()

# Retrieve all the nodes
nodes = {node.name: node for node in project.nodes}

# Define the IPv6 topology and configurations
topology_config = {
    "R1": {"g1/0": "2001:db8:1::1/64", "g2/0": "2001:db8:3::1/64"},
    "R2": {"g1/0": "2001:db8:1::2/64", "g3/0": "2001:db8:2::1/64"},
    "R3": {"g1/0": "2001:db8:3::2/64", "g2/0": "2001:db8:4::1/64"},
    "R4": {"g2/0": "2001:db8:2::2/64", "g3/0": "2001:db8:5::1/64", "f0/0": "2001:db8:6::1/64"},
    "R5": {"f0/0": "2001:db8:6::2/64", "g2/0": "2001:db8:7::1/64", "g3/0": "2001:db8:4::2/64"},
    "R6": {"g1/0": "2001:db8:5::2/64", "f0/0": "2001:db8:8::1/64"},
    "R7": {"f0/0": "2001:db8:8::2/64", "g2/0": "2001:db8:7::2/64"},
}

# Function to generate configuration file content
def generate_config(router_name, interfaces):
    config_lines = ["enable", "configure terminal"]
    # Add interface IPv6 configurations
    for interface, ipv6_address in interfaces.items():
        config_lines.append(f"interface {interface}")
        config_lines.append(f"ipv6 address {ipv6_address}")
        config_lines.append(f"ipv6 enable")
        config_lines.append("exit")
    # Enable RIPng
    config_lines.append("ipv6 router rip GNS3")
    for interface in interfaces.keys():
        config_lines.append(f"interface {interface}")
        config_lines.append("ipv6 rip GNS3 enable")
        config_lines.append("exit")
    # Save configuration
    config_lines.append("write memory")
    config_lines.append("exit")
    return "\n".join(config_lines)

# Apply configuration to each router
for router_name, interface_config in topology_config.items():
    # Get the router node
    node = nodes.get(router_name)
    if not node:
        print(f"Router {router_name} not found in the project.")
        continue
    
    # Generate the configuration file content
    config_content = generate_config(router_name, interface_config)
    config_filename = f"{router_name}_config.txt"

    # Save configuration to a file
    with open(config_filename, "w") as config_file:
        config_file.write(config_content)

    # Upload and run the configuration file on the router
    node.upload_and_run(config_filename)
    print(f"Configuration applied to {router_name} from {config_filename}.")

print("All configurations applied successfully!")


AttributeError: 'Node' object has no attribute 'upload_and_run'

In [5]:
import gns3fy
import time

# Connect to the GNS3 server
gns3_server = gns3fy.Gns3Connector("http://localhost:3080")

# Load the project
project = gns3fy.Project(name="GNS_v0", connector=gns3_server)
project.get()
project.open()

# Retrieve all the nodes
nodes = {node.name: node for node in project.nodes}

# Define the IPv6 topology and configurations
topology_config = {
    "R1": {"g1/0": "2001:db8:1::1/64", "g2/0": "2001:db8:3::1/64"},
    "R2": {"g1/0": "2001:db8:1::2/64", "g3/0": "2001:db8:2::1/64"},
    "R3": {"g1/0": "2001:db8:3::2/64", "g2/0": "2001:db8:4::1/64"},
    "R4": {"g2/0": "2001:db8:2::2/64", "g3/0": "2001:db8:5::1/64", "f0/0": "2001:db8:6::1/64"},
    "R5": {"f0/0": "2001:db8:6::2/64", "g2/0": "2001:db8:7::1/64", "g3/0": "2001:db8:4::2/64"},
    "R6": {"g1/0": "2001:db8:5::2/64", "f0/0": "2001:db8:8::1/64"},
    "R7": {"f0/0": "2001:db8:8::2/64", "g2/0": "2001:db8:7::2/64"},
}

# Function to send commands to a router via the console
def configure_router(router_name, interfaces):
    # Get the router node
    node = nodes.get(router_name)
    if not node:
        print(f"Router {router_name} not found.")
        return

    # Construct the configuration commands
    config_lines = ["enable", "configure terminal"]
    for interface, ipv6_address in interfaces.items():
        config_lines.append(f"interface {interface}")
        config_lines.append(f"ipv6 address {ipv6_address}")
        config_lines.append("ipv6 enable")
        config_lines.append("exit")
    config_lines.append("ipv6 router rip GNS3")
    for interface in interfaces.keys():
        config_lines.append(f"interface {interface}")
        config_lines.append("ipv6 rip GNS3 enable")
        config_lines.append("exit")
    config_lines.append("write memory")
    config_lines.append("exit")

    # Start the node if it's not running
    if not node.status == "started":
        node.start()
        time.sleep(5)  # Give time for the node to fully start

    # Open the console and send the configuration commands
    print(f"Configuring {router_name}...")
    try:
        output = node.console(lines=config_lines)
        print(f"Configuration applied to {router_name}:\n{output}")
    except Exception as e:
        print(f"Error configuring {router_name}: {e}")


# Apply configurations to all routers in the topology
for router_name, interface_config in topology_config.items():
    configure_router(router_name, interface_config)

print("All configurations applied successfully!")


Configuring R1...
Error configuring R1: 'int' object is not callable
Configuring R2...
Error configuring R2: 'int' object is not callable
Configuring R3...
Error configuring R3: 'int' object is not callable
Configuring R4...
Error configuring R4: 'int' object is not callable
Configuring R5...
Error configuring R5: 'int' object is not callable
Configuring R6...
Error configuring R6: 'int' object is not callable
Configuring R7...
Error configuring R7: 'int' object is not callable
All configurations applied successfully!
